In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMG_SIZE = 224
BATCH_SIZE = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/DAP_TEAM1/22A81A0505/Brain_Tumor_Detection/train',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/DAP_TEAM1/22A81A0505/Brain_Tumor_Detection/train',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation'
)

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    '/content/drive/MyDrive/DAP_TEAM1/22A81A0505/Brain_Tumor_Detection/pred',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary'
)
model = keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(train_generator,validation_data=val_generator,epochs=5)

model.save("Model.h5","label.txt")

from keras.models import load_model
from PIL import Image, ImageOps
import numpy as np

model = load_model('/content/Model.h5')

class_names = ['You have Brain Tumor','You do not have Brain Tumor']
data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)

image = Image.open("/content/drive/MyDrive/DAP_TEAM1/22A81A0505/Brain_Tumor_Detection/pred/pred0.jpg").convert("RGB")

size = (224, 224)
image = ImageOps.fit(image, size, Image.Resampling.LANCZOS)


image_array = np.asarray(image)

normalized_image_array = (image_array.astype(np.float32) / 127.5) - 1

data[0] = normalized_image_array

prediction = model.predict(data)
index = np.argmax(prediction)
confidence_score = prediction[0][index]

print("Result: ", class_names[index], end="")
print("\n")
print("Accuracy: ", confidence_score)


Found 2408 images belonging to 2 classes.
Found 601 images belonging to 2 classes.
Found 0 images belonging to 0 classes.
Epoch 1/5
76/76 [==============================] - 337s 4s/step - loss: 0.4496 - accuracy: 0.7998 - val_loss: 0.3411 - val_accuracy: 0.8403
Epoch 2/5
76/76 [==============================] - 342s 4s/step - loss: 0.2105 - accuracy: 0.9248 - val_loss: 0.1569 - val_accuracy: 0.9468
Epoch 3/5
76/76 [==============================] - 343s 5s/step - loss: 0.1031 - accuracy: 0.9684 - val_loss: 0.0589 - val_accuracy: 0.9784
Epoch 4/5
76/76 [==============================] - 340s 4s/step - loss: 0.0558 - accuracy: 0.9817 - val_loss: 0.0486 - val_accuracy: 0.9917
Epoch 5/5
1/1 [==============================] - 0s 131ms/step
Result:  You have Brain Tumor

Accuracy:  0.7258472
